In [ ]:
POP_SIZE = 12
GENERATIONS = 12
CROSSOVER_RATE = 0.8
MUTATION_RATE  = 0.04
ELITISM = 2

In [ ]:
def rand_feasible_placement():
    x = np.zeros((N, S), dtype=int)
    for n in range(N):
        perm = list(range(S))
        random.shuffle(perm)
        used = 0
        for s in perm:
            rs = services[s]['rs']
            if used + rs <= BS[n]['R']:
                if random.random() < 0.35:
                    x[n, s] = 1
                    used += rs
    repair_placement(x)
    return x

def placement_to_vector(x): return x.flatten()
def vector_to_placement(v): return v.reshape((N, S)).astype(int)

def compute_marginal_benefit_per_gb(bs_index, placement):
    benefit = {}
    for s in range(S):
        if placement[bs_index, s] == 0: continue
        count = 0.0
        for uid in users_by_service[s]:
            if bs_index in users[uid]['nearby']:
                other_hosts = [m for m in users[uid]['nearby']
                               if m != bs_index and placement[m, s] == 1]
                count += 1.0 if len(other_hosts)==0 else 0.2
        rs = services[s]['rs']
        benefit[s] = count / (rs + 1e-9)
    return benefit

def repair_placement(placement):
    for n in range(N):
        used = int(placement[n] @ np.array([services[s]['rs'] for s in range(S)]))
        if used <= BS[n]['R']: continue
        benefit = compute_marginal_benefit_per_gb(n, placement)
        placed = [s for s in range(S) if placement[n, s]==1]
        placed_sorted = sorted(placed, key=lambda s: benefit.get(s, 0.0))
        idx = 0
        while used > BS[n]['R'] and idx < len(placed_sorted):
            srm = placed_sorted[idx]
            placement[n, srm] = 0
            used -= services[srm]['rs']
            idx += 1
    return placement


In [ ]:
def evaluate_chromosome(vec):
    placement = vector_to_placement(np.array(vec))
    repair_placement(placement)
    _, edge_served, usage = aco_routing(placement)
    return edge_served, placement, usage

def tournament_selection(pop, fitnesses, k=3):
    idxs = random.sample(range(len(pop)), k)
    return pop[max(idxs, key=lambda i: fitnesses[i])]

def uniform_crossover(p1,p2):
    mask = np.random.rand(len(p1))<0.5
    child=p1.copy(); child[mask]=p2[mask]; return child

def mutate_vector(vec,rate=MUTATION_RATE):
    for i in range(len(vec)):
        if random.random()<rate: vec[i]=1-vec[i]
    return vec

def run_hybrid_ga_aco():
    population=[placement_to_vector(rand_feasible_placement()) for _ in range(POP_SIZE)]
    best_vec,best_fit=None,-1
    for gen in range(GENERATIONS):
        fitnesses=[]
        for indiv in population:
            fit,_,_=evaluate_chromosome(indiv)
            fitnesses.append(fit)
            if fit>best_fit: best_fit, best_vec=fit, indiv.copy()
        if gen%3==0 or gen==GENERATIONS-1:
            print(f"[Gen {gen}] best_fit={max(fitnesses)} mean_fit={np.mean(fitnesses):.2f}")
        ranked=sorted(list(zip(population,fitnesses)),key=lambda x:x[1],reverse=True)
        elites=[deepcopy(x[0]) for x in ranked[:ELITISM]]
        new_pop=elites.copy()
        while len(new_pop)<POP_SIZE:
            p1=tournament_selection(population,fitnesses)
            p2=tournament_selection(population,fitnesses)
            child=p1.copy()
            if random.random()<CROSSOVER_RATE: child=uniform_crossover(p1,p2)
            child=mutate_vector(child,MUTATION_RATE)
            child=placement_to_vector(repair_placement(vector_to_placement(child)))
            new_pop.append(child)
        population=new_pop
    return best_vec,best_fit
